In [3]:
import numpy as np
import cv2
from PIL import Image
import face_recognition


def crop(image):
    image = face_recognition.load_image_file(image)
    face_locations = face_recognition.face_locations(image)
    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = image[top:bottom, left:right]
        pil_image = Image.fromarray(face_image)
        pil_image.save('image/crop.jpg')

        
def color(img):
    avg = cv2.mean(img)
    print(avg)
    rec = np.zeros((384, 384,3), np.uint8)
    rec = cv2.rectangle(rec, (110, 110), (200, 200), avg, -1)
    cv2.imshow('avg', rec)
    cv2.imwrite("skin_color.jpg", rec)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
def extractSkin(image):
    img = image.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_threshold = np.array([0, 51, 80], dtype=np.uint8)
    upper_threshold = np.array([21, 153, 255], dtype=np.uint8)
    skinMask = cv2.inRange(img, lower_threshold, upper_threshold)
    skinMask = cv2.GaussianBlur(skinMask, (3, 3), 0)
    skin = cv2.bitwise_and(img, img, mask=skinMask)
    return cv2.cvtColor(skin, cv2.COLOR_HSV2BGR)


crop('image/soccer.jpg')
image =cv2.imread('image/crop.jpg')
image = imutils.resize(image, width=500)
skin = extractSkin(image)
cv2.imshow('skin', skin)
color(skin)

(79.32517434869739, 98.5569258517034, 135.64903006012023, 0.0)
